In [4]:
import numpy as np
from glob import glob

#We're using function from image_recognition to process photos
# from image_recognition import

#This is where the classification and categorization of images will occur
data_dir = 'Dataset'
train = 'Dataset/training'
test = 'Dataset/evaluation'
valid = 'Dataset/validation'
label_names = ['Bread', 'Dairy Product', 'Dessert', 'Egg', 'Fried Food', 'Meat', 'Noodles-Pasta', 'Rice',
               'Seafood', 'Soup', 'Vegetable-Fruit']

training_files = glob('Dataset/training/*/*.jpg')
training_set_size = [50,100, 250, 500, 1000, 5000, 9000]

for i in range(len(training_files)):
    print("Training_files array #: " +(str(i))+ training_files[i])
new_training_set = []
#This will split the training set into different sizes
"""
def split_training_size(training_files, n):
    for i in range(n):
        new_training_set.append(training_files[i])
    return new_training_set
"""


 

Dataset/training/Noodles-Pasta/63.jpg
Dataset/training/Noodles-Pasta/189.jpg
Dataset/training/Noodles-Pasta/77.jpg
Dataset/training/Noodles-Pasta/162.jpg
Dataset/training/Noodles-Pasta/176.jpg
Dataset/training/Noodles-Pasta/88.jpg
Dataset/training/Noodles-Pasta/348.jpg
Dataset/training/Noodles-Pasta/360.jpg
Dataset/training/Noodles-Pasta/406.jpg
Dataset/training/Noodles-Pasta/412.jpg
Dataset/training/Noodles-Pasta/374.jpg
Dataset/training/Noodles-Pasta/228.jpg
Dataset/training/Noodles-Pasta/214.jpg
Dataset/training/Noodles-Pasta/200.jpg
Dataset/training/Noodles-Pasta/201.jpg
Dataset/training/Noodles-Pasta/215.jpg
Dataset/training/Noodles-Pasta/229.jpg
Dataset/training/Noodles-Pasta/413.jpg
Dataset/training/Noodles-Pasta/375.jpg
Dataset/training/Noodles-Pasta/361.jpg
Dataset/training/Noodles-Pasta/407.jpg
Dataset/training/Noodles-Pasta/349.jpg
Dataset/training/Noodles-Pasta/177.jpg
Dataset/training/Noodles-Pasta/89.jpg
Dataset/training/Noodles-Pasta/163.jpg
Dataset/training/Noodles-Past

'\ndef split_training_size(training_files, n):\n    for i in range(n):\n        new_training_set.append(training_files[i])\n    return new_training_set\n'

In [25]:
import os
# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

datagen = ImageDataGenerator()
train_generator = datagen.flow_from_directory( 
    directory=train, 
    classes = label_names,
    target_size=(512, 512),  
    batch_size=50, 
    class_mode="binary", 
)
# validation data
valid_generator = datagen.flow_from_directory( 
    directory=valid, 
    classes = label_names,
    target_size=(512, 512), 
    batch_size=50, 
    class_mode="binary", 
    shuffle = True
)

    # build the entire model
    # Code taken from 
    # https://github.com/ovh/ai-training-examples/blob/main/notebooks/computer-vision/image-classification/tensorflow/resnet50/notebook-resnet-transfer-learning-image-classification.ipynb
    # Created by eleapttn
def model_creation():
    resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
    for layer in resnet_50.layers:
        layer.trainable = False
    x = resnet_50.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(11, activation='softmax')(x)
    return (Model(inputs = resnet_50.input, outputs = predictions))

# define training function
    # Code taken from 
    # https://github.com/ovh/ai-training-examples/blob/main/notebooks/computer-vision/image-classification/tensorflow/resnet50/notebook-resnet-transfer-learning-image-classification.ipynb
    # Created by eleapttn
def trainModel(model, passes, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, validation_data=valid_generator, epochs=passes, batch_size=batch_size)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [26]:
model = model_creation()
model_history = trainModel(model, 10, "Adam")

AttributeError: 'NoneType' object has no attribute 'compile'

In [22]:
test_data = datagen.flow_from_directory( 
    directory=test, 
    classes = label_names,
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="binary",
    shuffle=False
)


def find_accuracy(labels, predictions):
    sum = 0
    for i in range(len(labels)):
        if labels[i]==predictions[i]:
            sum +=1
    return sum/len(labels)


test_class = test_data.labels
print(test_class)
training_Accuracy = []
validation_Accuracy = []
testing_Accuracy = []
for set in training_sets:
    model = model_creation()
    model_history = trainModel(model, passes=10, optimizer="Adam")
    predictions = model.predict(test_data)
    class_ids = np.argmax(predictions, axis=1)
    training_Accuracy.append(find_accuracy(test_class, class_ids))
    validation_Accuracy.append(model_history.)
    

Found 3347 images belonging to 11 classes.
[ 0  0  0 ... 10 10 10]


AttributeError: 'NoneType' object has no attribute 'predict'